In [1]:
import sys

sys.path.append("..")
from typing import List

import requests
from bs4 import BeautifulSoup
from googlesearch import search
from pydantic import BaseModel, Field
from rich import print

from brain.sdk import BrainClient

In [2]:
brain_client = BrainClient("http://127.0.0.1:8000")

In [3]:
from typing import List, Union

from pydantic import BaseModel, Field


class Tool(BaseModel):
    script: str = Field(..., description="The Python script to be executed. Ensure it handles errors and outputs structured results.")
    pip_install: List[str] = Field(..., description="The list of pip packages required to execute the script.")
    completed: Union[bool, str] = Field(..., description="Indicates if the task is successfully completed. True if we are able to generate script, otherwise a string explaining why the script cannot be generated.")

In [4]:
@brain_client.reasoner(schema=Tool)
def autonomous_tool(task: str):
    """
    Generates standalone, executable Python scripts with no external dependencies.
    """
    system_prompt = """You are a Python script generator focused on creating fully autonomous code. Your scripts must be completely self-contained:

DO NOT generate scripts that:
- Require external inputs (API keys, file paths, user prompts, etc.)
- Use mock/placeholder data
- Do not Simulate or Mock artificial data
- Need manual configuration
- Access local files or environment variables
- Connect to external services without built-in test data
- Depend on system-specific resources
- Use unreliable or niche packages

Instead:
- Use standard library when possible
- Include test data within the script
- Handle all edge cases and errors
- Use only widely-used PyPI packages if needed
- Generate any required data programmatically
- If we cannot do it without external dependencies, explain why in 'completed' field and guide the tool maker to say it cannot be done

IMPORTANT:
**Do not simulate data or use mock data.**
**Do not use methods that require secrets or API keys or other blocking algorithms in script**
**If we cannot write the script with the above constraints, Fail and explain why in the 'completed' field.**
**if local file reference is given use approrpiate input path in the script to read the file or save the file etc..**

If task requires unavoidable external dependencies or simulated or mock data, explain why in 'completed' field and make it a fail case."""

    user_prompt = f"""Generate a standalone Python script for: "{task}"

Return:
- script: Fully executable Python code
- pip_install: Required packages (empty list if none needed)
- completed: True if successful, or explanation why task cannot be autonomous"""

    return system_prompt, user_prompt
autonomous_tool_id=autonomous_tool.register()

In [14]:
import os
import shutil
import subprocess
import tempfile
import venv
from typing import Dict, Union


def execute_script_with_temp_venv(script: str, pip_install: list) -> Dict[str, Union[str, bool]]:
    result = {
        "script": script,
        "pip_install": pip_install,
        "success": False,
        "stdout": "",
        "stderr": "",
        "error": ""
    }
    temp_dir = None

    try:
        # Step 1: Create a temporary directory for the virtual environment
        temp_dir = tempfile.mkdtemp()
        venv_dir = os.path.join(temp_dir, "venv")
        venv.create(venv_dir, with_pip=True)

        # Step 2: Paths for the virtual environment
        venv_python = os.path.join(venv_dir, "bin", "python") if os.name != 'nt' else os.path.join(venv_dir, "Scripts", "python")
        venv_pip = os.path.join(venv_dir, "bin", "pip") if os.name != 'nt' else os.path.join(venv_dir, "Scripts", "pip")

        # Step 3: Install required packages
        if pip_install:
            install_command = [venv_pip, "install"] + pip_install
            subprocess.run(install_command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Step 4: Write the script to a temporary file
        script_path = os.path.join(temp_dir, "script.py")
        with open(script_path, "w") as script_file:
            script_file.write(script)

        # Step 5: Execute the script
        execute_command = [venv_python, script_path]
        process = subprocess.run(
            execute_command, check=False, capture_output=True, text=True
        )

        # Step 6: Update result based on execution outcome
        result["stdout"] = process.stdout.strip()
        result["stderr"] = process.stderr.strip()
        result["error"] = ""
        result["success"] = process.returncode == 0 and not result["stderr"]

    except subprocess.CalledProcessError as e:
        result["error"] = f"Pip installation or execution failed: {str(e)}"
    except Exception as e:
        result["error"] = f"Unexpected error: {str(e)}"
    finally:
        # Step 7: Cleanup temporary directory
        if temp_dir and os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)

    # Update success condition to ensure no stderr or unexpected errors
    result["success"] = result["success"] and not result["error"] and not result["stderr"]
    return result

def format_tool_output(execution_result: Dict[str, Union[str, bool]]) -> str:
    if execution_result["success"]:
        return f"Tool executed successfully. Output:\n{execution_result['stdout']}\n"
    else:
        return (
            f"Tool execution failed.\n"
            f"Error: {execution_result['error'] if execution_result['error'] else 'No additional error details.'}\n"
            f"Standard Error: {execution_result['stderr']}\n"
            f"Standard Output: {execution_result['stdout']}\n"
            f"Script:\n{execution_result['script']}\n"
            f"Dependencies:\n{execution_result['pip_install']}\n"
        )

In [55]:
from typing import Optional, Union

from pydantic import BaseModel, Field


class Success(BaseModel):
    final_answer: str = Field(..., description="The final result or answer to the task if the execution succeeded.")

class Failure(BaseModel):
    analysis: str = Field(..., description="A detailed explanation of why the tool execution failed.")
    suggestions: str = Field(..., description="Suggestions for fixing the issue or alternative approaches to achieve the task.")

class Result(BaseModel):
    success: bool = Field(..., description="Indicates if the tool execution was successful.")
    result: Union[Success, Failure] = Field(..., description="The result of the analysis, including success or failure details.")
    
@brain_client.reasoner(schema=Result)
def result_analyzer(tool_output: str, task: str):
    """
    Analyzes script execution results focusing on zero human intervention requirement.
    """
    system_prompt = """Analyze Python script results for true autonomy - script must run without any human intervention.

Success case:
- Mention success as True
Extract meaningful result from tool_output and present as:
- Direct answer addressing the original task
- Natural, human-readable format
- No technical details unless relevant
- No mention of script/code aspects


Failure case - Check for:
1. Manual Requirements (like these, but it can be more):
   - API keys/credentials needed
   - User prompts/inputs
   - Configuration steps
   - Runtime user decisions
   - If this task cannot be done without mocking or simulating or manual intervention please mention clearly this as a main reason for failure and ask the tool to reply saying this cannot be done

Suggestions:
1. If failure due to manual steps, suggest truly autonomous alternatives
2. if the script uses API or mock data or synthetic data, recomend it to come up with a solution that can be done without these.
3. ensure the script is fully autonomous and can be run without any manual intervention or need for external api keys or manual inputs.
4. Ensure there are no Mock or synthetic data used in the script.

2. Script should:
   - Use standard library or pip packages
   - Make API calls if no credentials needed
   - Handle all data programmatically
   - Run start-to-finish autonomously
   - Should not mock or simulate any data or use placeholders
   - Should make sure we are achiving the task as expected without artificial hallucinated data.

Key principle:
If task requires ANY manual steps (API keys, inputs, etc):
- Clearly state why it can't be autonomous
- Don't suggest mock/placeholder solutions
- Only suggest truly automatic alternatives"""

    user_prompt = f"""Analyze result for: "{task}"
Output: "{tool_output}"

If tool output does not have any errors and tool has executed successfully, provide the final answer in a human-readable format.

Success: Direct task response
Failure: Explain manual intervention needs"""

    return system_prompt, user_prompt
result_analyzer_id = result_analyzer.register()

In [9]:
from typing import List, Optional
from pydantic import BaseModel, Field


class ToolExecutionResult(BaseModel):
    """
    Encapsulates the result of the autonomous tool's execution process.
    """
    final_result: Optional[str] = Field(None, description="The final result or answer to the task if successful.")
    final_script: Optional[str] = Field(None, description="The final generated Python script.")
    pip_dependencies: List[str] = Field(..., description="The list of pip dependencies required for the script.")
    execution_output: Optional[str] = Field(None, description="The output from executing the script.")
    success: bool = Field(..., description="Indicates if the task was successfully executed.")
    failure_reason: Optional[str] = Field(None, description="The reason for failure if the task could not be completed.")


@brain_client.multi_agent
def autonomous_tool(task: str, max_trials: int = 5, verbose: bool = True) -> ToolExecutionResult:
    """
    Multi-agent workflow to iteratively generate, execute, and analyze tools for a given task.

    Args:
        task (str): The task description.
        max_trials (int): Maximum number of attempts to generate and execute a successful tool.
        verbose (bool): Whether to print detailed logs.

    Returns:
        ToolExecutionResult: Structured result of the autonomous tool's execution.
    """
    for iteration in range(max_trials):
        print(f"{'=' * 10} Iteration {iteration + 1} {'=' * 10}")

        # Add context for retries if not the first iteration
        if iteration != 0:
            task_context = (
                f"Last tool failed because {tool_output.completed}. "
                f"Last tool script:\n{tool_output.script if tool_output.completed is True else 'N/A'}\n"
                f"Pip dependencies: {tool_output.pip_install if tool_output.completed is True else 'N/A'}\n"
            )
        else:
            task_context = ""

        # Step 1: Generate the tool
        tool_output = brain_client.use(autonomous_tool_id)(task=task_context + task)

        # If tool generation failed, return failure details
        if tool_output.completed is not True:
            if verbose:
                print(f"Tool Generation Failed: {tool_output.completed}")
            return ToolExecutionResult(
                final_result=None,
                final_script=None,
                pip_dependencies=[],
                execution_output=None,
                success=False,
                failure_reason=str(tool_output.completed)
            )

        # Step 2: Execute the generated script
        exec_output = execute_script_with_temp_venv(tool_output.script, tool_output.pip_install)

        # Step 3: Format execution result for analysis
        context = format_tool_output(exec_output)

        # Step 4: Analyze the result
        res = brain_client.use(result_analyzer_id)(tool_output=context, task=task)

        # Verbose logging for debugging
        if verbose:
            print(f"Result Analysis: {res.result}")

        # If successful, return the result details
        if res.success:
            if verbose:
                print(f"Final Tool Output:\n{context}")
            return ToolExecutionResult(
                final_result=res.result.final_answer,
                final_script=tool_output.script,
                pip_dependencies=tool_output.pip_install,
                execution_output=context,
                success=True
            )

    # If all attempts fail, return the failure details
    return ToolExecutionResult(
        final_result=None,
        final_script=tool_output.script if tool_output.completed is True else None,
        pip_dependencies=tool_output.pip_install if tool_output.completed is True else [],
        execution_output=None,
        success=False,
        failure_reason=res.result.analysis if 'res' in locals() and not res.success else "Unknown failure."
    )


In [10]:
task="What is the temperature in Downtown Toronto today?"
result=autonomous_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Result Analysis: analysis='The execution failed because the script relies on simulated data rather than fetching 
real-time data from an API. Real-time temperature data for Downtown Toronto cannot be obtained without API keys or 
credentials, making the task unachievable without manual intervention. The task cannot be performed autonomously.' 
suggestions='To achieve true autonomy, a new approach should involve using a public weather API that does not 
require authentication or credentials. The script should be modified to directly call such an API and handle the 
data programmatically, ensuring it can run without the need for any simulated or mocked data.'

========== Iteration 2 ==========

Tool Generation Failed: False

Final Result: final_result=None final_script=None pip_dependencies=[] execution_output=None success=False 
failure_reason='False'

In [11]:
task="search for top 3 articles on autonomous driving and give back the titles"
result=autonomous_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Result Analysis: analysis="The execution of the script failed to retrieve the top 3 articles on autonomous driving 
due to its reliance on an external web scraping approach that requires a stable internet connection and access to 
Google's search results page without any required API keys or manual input. Additionally, web scraping can often be
blocked or limited by the target websites, making it an unreliable method for autonomous tasks." suggestions='To 
achieve truly autonomous results without any manual intervention, consider using a reliable news API (such as 
NewsAPI or Bing News Search API) that does not require manual input or credentials for basic usage. This way, you 
can programmatically retrieve the latest articles on autonomous driving while ensuring full autonomy in the 
execution.'

========== Iteration 2 ==========

Tool Generation Failed: True

Final Result: final_result=None final_script=None pip_dependencies=[] execution_output=None success=False 
failure_reason='True'

In [15]:
task="calculate the sum of numbers from 1 to 100_0000_000"
result=autonomous_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Result Analysis: final_answer='The sum of numbers from 1 to 100000000 is: 5000000050000000'

Final Tool Output:
Tool executed successfully. Output:
The sum of numbers from 1 to 100000000 is: 5000000050000000

Final Result: final_result='The sum of numbers from 1 to 100000000 is: 5000000050000000' final_script="def 
sum_of_numbers(n):\n    return sum(range(1, n + 1))\n\nif __name__ == '__main__':\n    try:\n        n = 
100_000_000  # This is the upper limit as requested\n        result = sum_of_numbers(n)\n        print(f'The sum of
numbers from 1 to {n} is: {result}')\n    except Exception as e:\n        print(f'An error occurred: {e}')" 
pip_dependencies=[] execution_output='Tool executed successfully. Output:\nThe sum of numbers from 1 to 100000000 
is: 5000000050000000\n' success=True failure_reason=None

In [16]:
task="Multiply 777_82 times 67625_7762_8"
result=autonomous_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Result Analysis: final_answer='The multiplication of 77782 and 6762577628 results in 526006813061096.'

Final Tool Output:
Tool executed successfully. Output:
The result of multiplying 77782 and 6762577628 is: 526006813061096

Final Result: final_result='The multiplication of 77782 and 6762577628 results in 526006813061096.' 
final_script='def multiply_numbers(a, b):\n    """Multiply two numbers and return the result."""\n    return a * 
b\n\n# Test Data\nnum1 = 77782\nnum2 = 6762577628\n\ntry:\n    result = multiply_numbers(num1, num2)\n    
print(f\'The result of multiplying {num1} and {num2} is: {result}\')\nexcept Exception as e:\n    print(f\'An error
occurred: {e}\')' pip_dependencies=[] execution_output='Tool executed successfully. Output:\nThe result of 
multiplying 77782 and 6762577628 is: 526006813061096\n' success=True failure_reason=None

In [17]:
print(777_82*67625_7762_8)

526006813061096

In [18]:
task="Get ellon musk's latest tweet"
result=autonomous_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Tool Generation Failed: The task cannot be completed because accessing Elon Musk's latest tweet requires connecting
to the Twitter API, which necessitates using an API key and other authentication methods that fall outside the 
constraints provided. Therefore, it cannot be done without external dependencies or secret configurations.

Final Result: final_result=None final_script=None pip_dependencies=[] execution_output=None success=False 
failure_reason="The task cannot be completed because accessing Elon Musk's latest tweet requires connecting to the 
Twitter API, which necessitates using an API key and other authentication methods that fall outside the constraints
provided. Therefore, it cannot be done without external dependencies or secret configurations."

## Tool planner

In [56]:
from pydantic import BaseModel, Field
from typing import List

class Instructions(BaseModel):
    feasibility: str = Field(..., description="Analysis of whether the task can be completed autonomously, including any potential blockers")
    implementation_plan: str = Field(..., description="Detailed step-by-step plan for implementing the solution, including function breakdowns and data handling approaches")
    requirements: List[str] = Field(..., description="List of specific requirements including Python packages, error handling needs, and data processing requirements")
    execution_flow: str = Field(..., description="Precise description of how the script should execute, including data handling, function calls, and expected outcomes")
    is_autonomous: bool = Field(..., description="Whether the task can be completed without any human intervention, API keys, or manual human dependencies")

    

@brain_client.reasoner(schema=Instructions)
def feature_planner(task: str):
    """
    Generates detailed instructions for building autonomous Python scripts.
    """
    system_prompt = """You are a Python architecture planner specialized in designing autonomous scripts. Your role is to analyze tasks and provide clear, actionable instructions that ensure zero human intervention.

Analyze each task for true autonomy by considering:

Autonomy Requirements:
- Script must run without any API keys or credentials
- No user inputs can be required
- No environment variables or configuration needed
- No external service dependencies
- No system-specific resources

Input guidelines:
- make sure you specify what exact inputs are needed for the script to do the given task.

Implementation Guidelines:
- Prefer Python standard library solutions
- Use only common, stable PyPI packages if needed
- Include comprehensive error handling
- No mock or simulated data allowed
- No placeholder values
- Try to be Clever to come up with a solution that can be done without any blockers like apikey or manual input.
- We can have any pip packages that is needed and we can install it on the fly

Data Handling Rules:
- If task mentions file paths (e.g., "./something.something"), provide exact handling instructions
- Data must be programmatically generated or bundled
- No external data sources allowed if we cannot access them and needs api key or manual intervention
- No simulated or mock data

Your output should:
1. Assess if task can be truly autonomous
2. Provide detailed implementation steps
3. Specify exact execution flow
4. List concrete requirements
5. Flag any autonomy blockers

If the task cannot be autonomous:
- Set is_autonomous to false
- Explain specific blockers
- Do not suggest mock data workarounds
- Clearly state why task requires human intervention

Important: 
If any feedback is provided, make sure write your answer based on that feedback. Based on feedback update required packages or implementation plan etc..
If you think previous attempt failed because of some specific reason, make sure to either  address that in your answer or come up with a completely new solution based on the feedback provided to sovle the task exactly.
"""

    user_prompt = f"""Analyze this task and provide implementation instructions: "{task}"

Determine:
1. Is this task possible without any human intervention?
2. What exact implementation steps are needed?
3. How should data and paths be handled?
4. What specific requirements must be met?

Return Instructions with:
- Complete feasibility analysis
- Detailed implementation plan
- Specific requirements
- Precise execution flow
- Autonomy determination.
- Make sure to give detailed unambigious plan.

You Can run python scripts and install any pip packages without human intervention

Important: If any feedback is provided, make sure write your answer based on that feedback. Based on feedback update required packages or implementation plan etc.."""

    return system_prompt, user_prompt
feature_planner_id = feature_planner.register()

In [57]:
@brain_client.reasoner(schema=Tool)
def autonomous_planner_tool(instructions: Instructions):
    """
    Generates standalone Python scripts based on detailed implementation instructions.
    """
    system_prompt = """You are an expert Python script generator that creates fully autonomous code based on provided implementation instructions. Your goal is to generate scripts that run without any human intervention.
    
You Can run python scripts and install any pip packages without human intervention

Core Requirements:
1. Script Generation Rules:
   - No external inputs (API keys, credentials, user prompts)
   - No mock or simulated data
   - No manual configuration needs
   - No environment variables
   - No system-specific resources
   - No external service dependencies requiring authentication

2. Development Guidelines:
   - Prioritize Python standard library solutions
   - Use only widely-adopted PyPI packages when necessary
   - Implement comprehensive error handling
   - Generate data programmatically if needed
   - Handle all edge cases gracefully
   - Follow the provided implementation plan exactly
   - Use exact paths if specified in the implementation plan
   - We can have any pip packages that is needed and we can install it on the fly

3. Validation Requirements:
   - Ensure script runs without any user intervention
   - Verify no external dependencies are needed
   - Confirm no mock/simulated data is used
   - Follow execution flow from instructions
   - Implement all specified error handling

If task cannot be completed autonomously:
- Set completed to detailed explanation
- Do not attempt partial implementations
- Do not suggest mock data alternatives
- Explain specific blockers"""

    user_prompt = f"""Generate a Python script following these instructions:

Feasibility: {instructions.feasibility}
Implementation Plan: {instructions.implementation_plan}
Requirements: {instructions.requirements}
Execution Flow: {instructions.execution_flow}
Is Autonomous: {instructions.is_autonomous}

Return Tool schema with:
- Complete, autonomous Python script
- Required pip packages
- Success status or detailed explanation of blockers"""

    return system_prompt, user_prompt
autonomous_planner_tool_id = autonomous_planner_tool.register()

In [60]:
from typing import List, Optional
from pydantic import BaseModel, Field

class ToolExecutionResult(BaseModel):
    """
    Encapsulates the result of the autonomous tool's execution process.
    """
    final_result: Optional[str] = Field(None, description="The final result or answer to the task if successful.")
    final_script: Optional[str] = Field(None, description="The final generated Python script.")
    pip_dependencies: List[str] = Field(..., description="The list of pip dependencies required for the script.")
    execution_output: Optional[str] = Field(None, description="The output from executing the script.")
    success: bool = Field(..., description="Indicates if the task was successfully executed.")
    failure_reason: Optional[str] = Field(None, description="The reason for failure if the task could not be completed.")

@brain_client.multi_agent
def autonomous_planner_tool(task: str, max_trials: int = 5, verbose: bool = True) -> ToolExecutionResult:
    """
    Multi-agent workflow to plan, generate, execute, and analyze tools for a given task.
    """
    for iteration in range(max_trials):
        if verbose:
            print(f"{'=' * 10} Iteration {iteration + 1} {'=' * 10}")

        # Step 1: Generate implementation plan
        plan = brain_client.use(feature_planner_id)(task=task)
        
        # If task is not autonomous based on plan, return failure
        if not plan.is_autonomous:
            return ToolExecutionResult(
                final_result=None,
                final_script=None,
                pip_dependencies=[],
                execution_output=None,
                success=False,
                failure_reason=plan.feasibility
            )

        # Step 2: Generate tool based on plan
        tool_output = brain_client.use(autonomous_planner_tool_id)(instructions=plan)

        # If tool generation failed, continue to next iteration
        if tool_output.completed is not True:
            if verbose:
                print(f"Tool Generation Failed: {tool_output.completed}")
            continue

        # Step 3: Execute the generated script
        exec_output = execute_script_with_temp_venv(tool_output.script, tool_output.pip_install)

        # Step 4: Format execution result for analysis
        context = format_tool_output(exec_output)

        # Step 5: Analyze the result
        res = brain_client.use(result_analyzer_id)(tool_output=context, task=task)

        if verbose:
            print(f"Result Analysis: {res.result}")

        # If successful, return the result details
        if res.success:
            if verbose:
                print(f"Final Tool Output:\n{context}")
            return ToolExecutionResult(
                final_result=res.result.final_answer,
                final_script=tool_output.script,
                pip_dependencies=tool_output.pip_install,
                execution_output=context,
                success=True
            )

        # If failed, add context for next iteration
        task = (
            f"Previous Implementation plan:\n{plan.implementation_plan}"
            f"Previous attempt failed.\nAnalysis: {res.result.analysis}\n"
            f"Suggestions: {res.result.suggestions}\n"
            f"Original task: {task}\n"
        )

    # If all attempts fail, return the failure details
    return ToolExecutionResult(
        final_result=None,
        final_script=tool_output.script if tool_output.completed is True else None,
        pip_dependencies=tool_output.pip_install if tool_output.completed is True else [],
        execution_output=None,
        success=False,
        failure_reason=res.result.analysis if 'res' in locals() and not res.success else "Maximum iterations reached without success."
    )

In [22]:
task="Extract the top 5 most frequent words from the text file ./input.txt and ignore common stopwords."
result=autonomous_planner_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Result Analysis: final_answer='The task was executed successfully, and the top 5 most frequent words found in the 
text file are:\n\n1. may: 1664\n2. _: 1064\n3. acid: 734\n4. death: 728\n5. cases: 549\n\nThis analysis effectively
disregards common stopwords as required.'

Final Tool Output:
Tool executed successfully. Output:
Top 5 words:
may: 1664
_: 1064
acid: 734
death: 728
cases: 549

Final Result: final_result='The task was executed successfully, and the top 5 most frequent words found in the text
file are:\n\n1. may: 1664\n2. _: 1064\n3. acid: 734\n4. death: 728\n5. cases: 549\n\nThis analysis effectively 
disregards common stopwords as required.' final_script='import collections\nimport re\nimport os\n\n# Function to 
load the text file\n\ndef load_file(file_path):\n    try:\n        if not os.path.isfile(file_path):\n            
raise FileNotFoundError(f"File not found: {file_path}")\n        with open(file_path, \'r\', encoding=\'utf-8\') as
file:\n            content = file.read()\n        return content\n    except Exception as e:\n        return 
str(e)\n\n# List of common stopwords\nstopwords = {\n    \'i\', \'me\', \'my\', \'myself\', \'we\', \'our\', 
\'ours\', \'ourselves\',\n    \'you\', \'your\', \'yours\', \'yourself\', \'yourselves\', \'he\', \'him\',\n    
\'his\', \'himself\', \'she\', \'her\', \'hers\', \'herself\', \'it\', \'its\',\n    \'itself\', \'they\', 
\'them\', \'their\', \'theirs\', \'themselves\', \'what\',\n    \'which\', \'who\', \'whom\', \'this\', \'that\', 
\'these\', \'those\', \'am\', \'is\', \'are\',\n    \'was\', \'were\', \'be\', \'been\', \'being\', \'have\', 
\'has\', \'had\', \'having\',\n    \'do\', \'does\', \'did\', \'doing\', \'a\', \'an\', \'the\', \'and\', \'but\', 
\'if\',\n    \'or\', \'because\', \'as\', \'until\', \'while\', \'of\', \'at\', \'by\', \'for\',\n    \'with\', 
\'about\', \'against\', \'between\', \'into\', \'through\', \'during\',\n    \'before\', \'after\', \'above\', 
\'below\', \'to\', \'from\', \'up\', \'down\',\n    \'in\', \'out\', \'on\', \'off\', \'over\', \'under\', 
\'again\', \'further\',\n    \'then\', \'once\', \'here\', \'there\', \'when\', \'where\', \'why\', \'how\',\n    
\'all\', \'any\', \'both\', \'each\', \'few\', \'more\', \'most\', \'other\',\n    \'some\', \'such\', \'no\', 
\'nor\', \'not\', \'only\', \'own\', \'same\',\n    \'so\', \'than\', \'too\', \'very\', \'s\', \'t\', \'can\', 
\'will\', \'just\',\n    \'don\', \'should\', \'now\'\n}\n\n# Function to process the text\n\ndef 
process_text(content):\n    # Tokenize and clean the text\n    words = re.findall(r\'\\b\\w+\\b\', 
content.lower())\n    # Filter out stopwords\n    filtered_words = \n    # Count frequencies\n    word_counts = 
collections.Counter(filtered_words)\n    return word_counts\n\n# Function to extract the top 5 words\n\ndef 
get_top_words(word_counts, num=5):\n    return word_counts.most_common(num)\n\n# Main execution flow\nif __name__ 
== \'__main__\':\n    file_path = \'./input.txt\'\n    content = load_file(file_path)\n    if isinstance(content, 
str) and content.startswith(\'File not found\'):\n        print(content)\n    else:\n        word_counts = 
process_text(content)\n        if not word_counts:\n            print("No valid words found.")\n        else:\n    
top_words = get_top_words(word_counts)\n            print("Top 5 words:")\n            for word, count in 
top_words:\n                print(f\'{word}: {count}\')' pip_dependencies=[] execution_output='Tool executed 
successfully. Output:\nTop 5 words:\nmay: 1664\n_: 1064\nacid: 734\ndeath: 728\ncases: 549\n' success=True 
failure_reason=None

In [ ]:
# get absolute path of a file
file_path="./input.txt"
task=f"use plotly and plot the distribution of word length in the text file {file_path} and fit a nice curve to the distribution and save the plot as file."
result=autonomous_planner_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Result(
    success=False,
    result=Failure(
        analysis="The tool execution failed because the script requires the installation of an additional package 
('kaleido') to save the plot as an image. This requirement necessitates manual intervention to install the package 
using pip, which contradicts the principle of true autonomy.",
        suggestions='To achieve true autonomy, it is recommended to bundle the required dependencies with the 
script or ensure the script checks and installs any missing packages automatically. Alternatively, consider using a
different plotting library that does not require external installations for saving images.'
    )
)
Tool execution failed.
Error: No additional error details.
Standard Error: Traceback (most recent call last):
  File "/var/folders/j7/59f75dnd6yv58szy8857mfp80000gn/T/tmpcv4aewcp/script.py", line 39, in <module>
    fig.write_image('word_distribution.png')
  File 
"/var/folders/j7/59f75dnd6yv58szy8857mfp80000gn/T/tmpcv4aewcp/venv/lib/python3.10/site-packages/plotly/basedatatype
s.py", line 3835, in write_image
    return pio.write_image(self, *args, **kwargs)
  File 
"/var/folders/j7/59f75dnd6yv58szy8857mfp80000gn/T/tmpcv4aewcp/venv/lib/python3.10/site-packages/plotly/io/_kaleido.
py", line 266, in write_image
    img_data = to_image(
  File 
"/var/folders/j7/59f75dnd6yv58szy8857mfp80000gn/T/tmpcv4aewcp/venv/lib/python3.10/site-packages/plotly/io/_kaleido.
py", line 132, in to_image
    raise ValueError(
ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido
Standard Output: 
Script:
import os
import plotly.graph_objects as go
from collections import Counter

# Step 1: Read the contents of './input.txt'
input_file_path = './input.txt'

try:
    # Check if file exists
    if not os.path.isfile(input_file_path):
        raise FileNotFoundError(f'{input_file_path} does not exist.')
    
    # Read file content
    with open(input_file_path, 'r', encoding='utf-8') as file:
        content = file.read().strip()  # Remove any leading/trailing whitespace
    
    # Check if the content is empty
    if not content:
        raise ValueError(f'{input_file_path} is empty.')
except (FileNotFoundError, ValueError) as e:
    print(e)
    exit(1)

# Step 3: Count word frequencies, converting to lowercase
words = content.lower().split()  # Split into words
word_counts = Counter(words)  # Create frequency dictionary

# Step 4: Identify the top 10 most frequent words
most_common_words = word_counts.most_common(10)
words, counts = zip(*most_common_words)  # Unzip into two lists

# Step 5: Use plotly to create a bar chart visualizing the distribution
fig = go.Figure(data=)
fig.update_layout(title='Top 10 Most Frequent Words',
                  xaxis_title='Words',
                  yaxis_title='Frequency')

# Step 6: Save the plot as an image file
fig.write_image('word_distribution.png')

# Step 7: Print a success message
print('Bar chart saved successfully as word_distribution.png')
Dependencies:
['plotly']

Result Analysis: analysis="The tool execution failed because the script requires the installation of an additional 
package ('kaleido') to save the plot as an image. This requirement necessitates manual intervention to install the 
package using pip, which contradicts the principle of true autonomy." suggestions='To achieve true autonomy, it is 
recommended to bundle the required dependencies with the script or ensure the script checks and installs any 
missing packages automatically. Alternatively, consider using a different plotting library that does not require 
external installations for saving images.'

========== Iteration 2 ==========

Result(
    success=False,
    result=Failure(
        analysis="The tool execution failed because the script requires the installation of additional packages, 
such as 'kaleido' and 'pandas', to function properly. This need for package installation introduces manual steps, 
which contradicts the principle of true autonomy. The script will not operate independently without these 
installations, and no automatic package installation was successfully carried out, particularly when 'pandas' was 
not recognized as installed during execution.",
        suggestions='To achieve true autonomy, the script should be modified to either bundle the required 
dependencies or check and install all necessary packages automatically. It is also advisable to select a plotting 
library that does not demand additional installations for saving images, ensuring it can operate without external 
dependencies.'
    )
)
Tool execution failed.
Error: No additional error details.
Standard Error: Traceback (most recent call last):
  File "/var/folders/j7/59f75dnd6yv58szy8857mfp80000gn/T/tmph46lrmk_/script.py", line 3, in <module>
    import plotly.express as px
  File 
"/var/folders/j7/59f75dnd6yv58szy8857mfp80000gn/T/tmph46lrmk_/venv/lib/python3.10/site-packages/plotly/express/__in
it__.py", line 9, in <module>
    raise ImportError(
ImportError: Plotly express requires pandas to be installed.
Standard Output: 
Script:
import os
import subprocess
import plotly.express as px
from collections import Counter

# Step 1: Check and install necessary packages
required_packages = ['plotly', 'kaleido']

for package in required_packages:
    subprocess.check_call(["python", "-m", "pip", "install", package])

# Step 2: Read the contents of './input.txt'
file_path = './input.txt'

try:
    with open(file_path, 'r') as file:
        content = file.read()
except FileNotFoundError:
    print(f'Error: The file {file_path} was not found.')
    exit(1)

# Step 3: Check for empty content
if not content.strip():
    print('Error: The file is empty.');
    exit(1)

# Step 4: Convert text to lowercase and split into words
words = content.lower().split()

# Step 5: Count word frequencies
word_counts = Counter(words)

# Step 6: Extract top 10 most frequent words
top_words = word_counts.most_common(10)

# Prepare data for plotting
words_list, counts_list = zip(*top_words)

# Step 7: Generate a bar chart using plotly
fig = px.bar(x=words_list, y=counts_list, labels={'x':'Words', 'y':'Frequency'}, title='Top 10 Most Frequent 
Words')

# Step 8: Save the plot as an image file
try:
    fig.write_image('word_distribution.png')
except Exception as e:
    print(f'Error while saving the plot: {e}')
Dependencies:
['plotly', 'kaleido']

...

========== Iteration 5 ==========

Result(
    success=True,
    result=Success(
        final_answer="The analysis reveals that the execution of the task was successful. The script successfully 
identified the ten most common words in the text file and generated a visual representation in the form of a bar 
chart. The most common words are: \n- 'the': 15233 occurrences\n- 'of': 9520 occurrences\n- 'and': 5128 
occurrences\n- 'in': 4681 occurrences\n- 'to': 3806 occurrences\n- 'a': 3670 occurrences\n- 'is': 2996 
occurrences\n- 'be': 2630 occurrences\n- 'or': 2170 occurrences\n- '|': 1876 occurrences.\nThe bar chart depicting 
this word distribution has been saved as 'word_distribution.png'."
    )
)
Tool executed successfully. Output:
Most common words: [('the', 15233), ('of', 9520), ('and', 5128), ('in', 4681), ('to', 3806), ('a', 3670), ('is', 
2996), ('be', 2630), ('or', 2170), ('|', 1876)]
Bar chart saved as word_distribution.png

Result Analysis: final_answer="The analysis reveals that the execution of the task was successful. The script 
successfully identified the ten most common words in the text file and generated a visual representation in the 
form of a bar chart. The most common words are: \n- 'the': 15233 occurrences\n- 'of': 9520 occurrences\n- 'and': 
5128 occurrences\n- 'in': 4681 occurrences\n- 'to': 3806 occurrences\n- 'a': 3670 occurrences\n- 'is': 2996 
occurrences\n- 'be': 2630 occurrences\n- 'or': 2170 occurrences\n- '|': 1876 occurrences.\nThe bar chart depicting 
this word distribution has been saved as 'word_distribution.png'."

Final Tool Output:
Tool executed successfully. Output:
Most common words: [('the', 15233), ('of', 9520), ('and', 5128), ('in', 4681), ('to', 3806), ('a', 3670), ('is', 
2996), ('be', 2630), ('or', 2170), ('|', 1876)]
Bar chart saved as word_distribution.png

Final Result: final_result="The analysis reveals that the execution of the task was successful. The script 
successfully identified the ten most common words in the text file and generated a visual representation in the 
form of a bar chart. The most common words are: \n- 'the': 15233 occurrences\n- 'of': 9520 occurrences\n- 'and': 
5128 occurrences\n- 'in': 4681 occurrences\n- 'to': 3806 occurrences\n- 'a': 3670 occurrences\n- 'is': 2996 
occurrences\n- 'be': 2630 occurrences\n- 'or': 2170 occurrences\n- '|': 1876 occurrences.\nThe bar chart depicting 
this word distribution has been saved as 'word_distribution.png'." final_script='import importlib\nimport 
sys\nimport os\nimport collections\n\n# Function to install packages\ndef install(package):\n    try:\n        
importlib.import_module(package)\n    except ImportError:\n        import subprocess\n        
subprocess.check_call()\n\n# List of required packages\nrequired_packages = [\'pandas\', \'plotly\', 
\'kaleido\']\n\n# Install missing packages\nfor package in required_packages:\n    install(package)\n\ntry:\n    # 
Step 2: Read the contents of the \'./input.txt\' file\n    with open(\'./input.txt\', \'r\') as file:\n        text
= file.read().strip()\n        if not text:\n            raise ValueError(\'The file is empty.\')\nexcept 
FileNotFoundError:\n    print("Error: The file \'./input.txt\' was not found.")\n    sys.exit(1)\nexcept IOError as
e:\n    print(f"Error reading the file: {e}")\n    sys.exit(1)\nexcept ValueError as ve:\n    print(ve)\n    
sys.exit(1)\n\n# Step 3: Process the text\nwords = text.lower().split()\n\n# Step 4: Create a frequency 
dictionary\nword_counts = collections.Counter(words)\n\n# Step 5: Identify the 10 most frequent 
words\nmost_common_words = word_counts.most_common(10)\nprint(\'Most common words:\', most_common_words)\n\n# Step 
6: Generate a bar chart using \'plotly\'\nimport pandas as pd\nimport plotly.express as px\n\n# Prepare a 
DataFrame\ndf = pd.DataFrame(most_common_words, columns=[\'Word\', \'Frequency\'])\n\n# Create the bar chart\nfig =
px.bar(df, x=\'Word\', y=\'Frequency\', title=\'Top 10 Most Frequent Words\')\n\n# Step 7: Save the plot as 
\'word_distribution.png\'\ntry:\n    fig.write_image(\'word_distribution.png\')\nexcept Exception as e:\n    
print(f"Error saving the image: {e}")\n    sys.exit(1)\n\nprint(\'Bar chart saved as word_distribution.png\')\n' 
pip_dependencies=['pandas', 'plotly', 'kaleido'] execution_output="Tool executed successfully. Output:\nMost common
words: [('the', 15233), ('of', 9520), ('and', 5128), ('in', 4681), ('to', 3806), ('a', 3670), ('is', 2996), ('be', 
2630), ('or', 2170), ('|', 1876)]\nBar chart saved as word_distribution.png\n" success=True failure_reason=None

In [ ]:
task="search for top 3 articles on autonomous driving and give back the titles"
result=autonomous_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")